In [ ]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [ ]:
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
    losses,
    evaluation
)
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
import ast

In [ ]:
def load_triplet_data(path):
    df = pd.read_csv(path)
    df['negative'] = df['negatives'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Exploding the 'negative' column to create multiple rows
    df_expanded = df.explode('negative').reset_index(drop=True)

    return Dataset.from_dict({
        "anchor": df_expanded['anchor'].tolist(),
        "positive": df_expanded['positive'].tolist(),
        "negative": df_expanded['negative'].tolist()
    })

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/data/'


train_data = load_triplet_data(data_path + "train.csv")
test_data = load_triplet_data(data_path + "test.csv")
val_data = load_triplet_data(data_path + "dev.csv")

In [ ]:
train_data[0]

{'anchor': 'የታክስ ገቢ ከ2010-2012 በመቶኛ የምን ያህል መጠን እድገት አሳየ?',
 'positive': 'ጠቅላይ ሚኒስትር ዐቢይ አሕመድ ከ2010 ጀምሮ በፋይናንሱ ዘርፍ ስኬታማ ለውጦች መመዝገባቸውን ገለጹ፡፡ ጠቅላይ ሚኒስትር ዐቢይ የፋይናንስ ዘርፍ ዐበይት ስኬቶች በሚል በማህበራዊ ትስስር ገፃቸው ላይ እንዳስታወቁት የታክስ ገቢ በ2010 ከነበረበት 229 ቢሊየን ብር በ2012 የ36 በመቶ ጭማሪ በማሳየት 311 ቢሊየን ማድረስ ተችሏል።',
 'negative': 'ፋሲለደስ ዓፄ ፋሲለደስ ወይም ዓፄ ፋሲል (የዙፋን ስማቸው ዓለም ሰገድ)  ከአባታቸው አፄ ሱሰኒዮስ  እና እናታቸው ልዕልት ስልጣነ ምገሴ  በመገዛዝ፣ ሸዋ ህዳር 10፣ 1603  (እ.ኤ.አ) ተወለዱ። የነገሱበትም ዘመን ከ1632  እስከ ጥቅምት 18, 1667 (እ.ኤ.አ) ነበር። በስረፀ ክርስቶስ በተመራው አመፅ ምክንያት በ1630 ፋሲለደስ ለንግስና ቢበቃም፣ ዘውዱን ግን እስከ 1632 አልጫነም ነበር። ሲመተ ንግስናው በ1632 እንደተገባደደ የመጀመሪያው ስራው የተዋህዶ ቤ/ክርስቲያንን የቀድሞው ቁመና መመለስና የካቶሊኮችን መሬት በመቀማት ከደንካዝ በማባረር በፍሪሞና እንዲወሰኑ ማድረግ ነበር። ወዲያውም በማከታተል ከግብፅ አገር አዲስ ጳጳስ እንዲላክለት በማድረግ በአባቱ ዘመን እንዲደበዝዝ ተደርጎ የነበረውን የግብፅና ኢትዮጵያ አብያተ ክርስቲያናት ግንኙነት እንዲጸና አደረገ።  በኬኒያ የሚገኘው የሞምባሳ ወደብ በፖርቱጋሎች መደብደቡን ሲሰማ፣ የሮማው ፓፓ ከበስትጀርባው ያለበት ሴራ ነው በማለት በምድሩ የነበሩትን የካቶሊክ ጀስዊቶች በመሰብሰብ አባረራቸው። አፄ ፋሲል አዘዞ ተብላ በምትታወቀው ከጎንደር ከተማ 5 ማይል ርቃ በምትገኘው ከተማ ጥቅምት18፣1667 እ.ኤ.አ. (ጥቅምት 10፣ 1660) ከዚህ

In [ ]:
import random

def shuffle_dataset(dataset):
  """Shuffles the given dataset in-place."""
  items = list(dataset)
  random.shuffle(items)
  return Dataset.from_list(items)

train_data = shuffle_dataset(train_data)
test_data = shuffle_dataset(test_data)
val_data = shuffle_dataset(val_data)
train_data[0]

{'anchor': 'አፄ ይስሐቅ ያሰሩት ደብረ ይሥሓቅ የሚባለው ቤተክርስቲያን በየት ተሰራ?',
 'positive': 'አፄ ይስሐቅ ቀዳማዊ  አጼ ይሥሓቅ በዙፋን ስማቸው "ዳግማዊ ገብረ መስቀል" ሲባሉ እ.ኤ.አ ከ1414-1429 የኢትዮጵያ ንጉሠ ነገሥት የነበሩ ናቸው። የቀዳማዊ አጼ ዳዊት ሁለተኛ ወንድ ልጅ ሲሆኑ የቀዳማዊ ቴዎድሮስ ታናሽ ወንድም ናቸው። ከአክሱም ነገሥታት በኋላ የመጀመሪያውን ከአውሮጳውያን መሪወች ጋር ግንኙነት ያደረገው ይሄው ንጉስ ነበር። ለምሳሌ ለአራጎን መሪ ለነበርው አልፎንሶ አምስተኛ በ1428 ዓ.ም. በእስላሞች ላይ የተባበረ ሃይልን ለመመስረት ደብዳቤ ልኳል። በዚሁ ደብዳቤ የጋብቻን ነገር ያወሳ ሲሆን  ህጻኑ ዶን ፔድሮ  ከእጅ ጥበብ አዋቂወች ጋር ወደ ኢትዮጵያ መጥቶ የይስሓቅን ሴት ልጅ እንዲያገባ ያትታል።  አልፎንሶ ለይስሓቅ የመለሰለት ደብዳቤ ባይገኝም ለይስሓቅ ተከታ ለአጼ ዘርዓ ያዕቆብ በ1450 በደረሰ ደብዳቤ ደህንነታቸው እስከተጥበቀ ድረስ እጅ ጥበበኞችን በደስታ ወደኢትዮጵያ እንደሚልክና ከአሁን በፊት የላካቸው 13 ጥበበኞቹ በመንገድ ላይ እንደጠፉ ሳይጠቅስ አላለፈም። በኒሁ ንጉስ ዘመን በፈላሾች በተነሳ አመጽ ምክንያት ንጉሱ ወደወገራ ዘምተው አመጸኞቹን ኮሶጌ ላይ በማሸነፍ አመጹን አረገቡ። በዚያውም ደብረ ይሥሓቅ የተሰኘውን ቤተክርስቲያን ለድሉ ማስታወሻ ኮሶጌ ላይ አሰሩ። . ንጉሱ ተመልሰው ከአገው ምድር ባሻገር ያለውን የሻንቅላ ምድር ላይ ዘመቻ አካሂደዋል። በደቡብ ምስራቅ ደግሞ ከአረብ አገር የተመለሱትን የሳድ አዲን ፪ኛ ልጆች ወግተዋል። ታሪክ አጥኝው ታደሰ ታምራት ይስሓቅ ከመስሊሞች ጋር ሲዋጋ ወደቀ ይበል እንጂ ዋሊስ በድጅ በርግጥም በወንጀል እንደተገደለና በተድባባ ማርያም እንደተቀበረ ይናገራል።',
 'negative':

In [ ]:
# Option 1: Base XLM-R with PEFT

# model_name = "sentence-transformers/static-similarity-mrl-multilingual-v1"
# model_name= "sentence-transformers/multi-qa-mpnet-base-cos-v1" - FAILURE
model_name = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"
# model_name = "Pyjay/sentence-transformers-multilingual-snli-v2-500k" - Works but is too big

model = SentenceTransformer(
    model_name,
    model_card_data=SentenceTransformerModelCardData(
        language="am",
        license="MIT",
        model_name=f"{model_name} adapter finetuned on Amharic QA triplets",
    ),)



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
sample_text = "ውክፔዲያ ዓለምዓቀፍ የዕውቀት ማከማቻ ድረ-ገጽ ነው። ውክፔዲያ የጋራ ነው።"
tokens = model.tokenizer.tokenize(sample_text)
print("Tokenization:", tokens) # We might need to improve this later on, but it's working for now

Tokenization: ['▁', 'ው', 'ክ', 'ፔ', 'ዲያ', '▁ዓለም', 'ዓ', 'ቀ', 'ፍ', '▁የ', 'ዕውቀት', '▁ማ', 'ከ', 'ማ', 'ቻ', '▁ድረ', '-', 'ገ', 'ጽ', '▁ነው።', '▁', 'ው', 'ክ', 'ፔ', 'ዲያ', '▁የጋራ', '▁ነው።']


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def evaluate_model(model, model_name, anchor, candidates):
    print(f"\n{'='*30} {model_name} {'='*30}")
    anchor_emb = model.encode([anchor])
    candidate_embs = model.encode(candidates)

    # Calculate similarities
    similarities = cosine_similarity(anchor_emb, candidate_embs)[0]

    # Print ranked results
    ranked_indices = np.argsort(-similarities)
    print("Original Sentence: ", anchor)
    print("Sentence | Similarity Score")
    for idx in ranked_indices:
        print(f"{candidates[idx]} |  {similarities[idx]:.3f} ")

In [ ]:
query = "ሰማይ ምን አይነት ቀለም ነው?"
docs = [
    "ሰማይ ሰማያዊ ቀለም አለው።" ,
    "እኔ ምሳ እንጀራ በላሁ።" ,
    "ባሕር ምን አይነት ቀለም ነው?",
    "አየር በምድር ዙሪያ ያለ ነው።"
]

evaluate_model(model, "Original Model Performance: Sentence Similarity", query, docs)


============================== Original Model Performance: Sentence Similarity ==============================
Original Sentence:  ሰማይ ምን አይነት ቀለም ነው?
Sentence | Similarity Score
ባሕር ምን አይነት ቀለም ነው? |  0.764 
ሰማይ ሰማያዊ ቀለም አለው። |  0.540 
አየር በምድር ዙሪያ ያለ ነው። |  0.442 
እኔ ምሳ እንጀራ በላሁ። |  0.126 


In [ ]:
loss = losses.MatryoshkaLoss(
    model,
    losses.TripletLoss(model, triplet_margin=0.5),
    matryoshka_dims=[768, 512, 256],
    matryoshka_weights=[0.5, 0.4, 0.2]
  )

evaluator = evaluation.TripletEvaluator(
    anchors=val_data["anchor"],
    positives=val_data["positive"],
    negatives=val_data["negative"],
)

sts_evaluator = evaluation.EmbeddingSimilarityEvaluator(
    sentences1=val_data["anchor"],
    sentences2=val_data["positive"],
    scores=[1.0]*len(val_data),  # Assume perfect similarity
    main_similarity=evaluation.SimilarityFunction.COSINE
)


In [ ]:
args = SentenceTransformerTrainingArguments(
    output_dir="amharic-retrieval-ft",
    num_train_epochs=3,
    per_device_train_batch_size=22,  # Leverages T4's 16GB VRAM
    fp16=True,
    learning_rate=1e-5,
    warmup_ratio=0.08,
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="epoch",
    report_to="tensorboard",
)

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    loss=loss,
    evaluator=evaluation.SequentialEvaluator([sts_evaluator, evaluator]),
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Pearson Cosine,Spearman Cosine,Cosine Accuracy,Sequential Score
500,0.022400,0.163294,nan,nan,0.922222,0.922222


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.11/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:206: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_pearson, _ = pearsonr(labels, scores)
/usr/local/lib/python3.11/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:207: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_spearman, _ = spearmanr(labels, scores)


TrainOutput(global_step=918, training_loss=0.04720158654230612, metrics={'train_runtime': 576.5941, 'train_samples_per_second': 34.938, 'train_steps_per_second': 1.592, 'total_flos': 0.0, 'train_loss': 0.04720158654230612, 'epoch': 3.0})

In [ ]:
from sentence_transformers import evaluation

# Create a TripletEvaluator for the test set
test_evaluator = evaluation.TripletEvaluator(
    anchors=test_data["anchor"],
    positives=test_data["positive"],
    negatives=test_data["negative"],
    name="TestTripletEvaluator"
)

# Evaluate the model on the test dataset
test_score = test_evaluator(model)
print(f"Test Triplet Evaluation Score: {test_score['TestTripletEvaluator_cosine_accuracy']:.4f}")

Test Triplet Evaluation Score: 0.8861


In [ ]:
model.save("model/am-xlmr-ft-v-2.0.0")

In [ ]:
# prompt: copy "model/am-xlmr-ft-v-1.0" to my drive

import shutil
import os

# Assuming 'model/am-xlmr-ft-v-1.0' exists in your current Colab environment
source_dir = "model/am-xlmr-ft-v-2.0.0"
destination_dir = "/content/drive/MyDrive/am-xlmr-ft-v-2.0.0"  # Specify the destination path in your Google Drive

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

try:
  shutil.copytree(source_dir, destination_dir)
  print(f"Successfully copied '{source_dir}' to '{destination_dir}'")
except FileExistsError:
  print(f"Destination directory '{destination_dir}' already exists.  Skipping copy.")
except FileNotFoundError:
    print(f"Source directory '{source_dir}' not found. Please ensure the model directory exists.")
except Exception as e:
  print(f"An error occurred: {e}")

Destination directory '/content/drive/MyDrive/am-xlmr-ft-v-2.0.0' already exists.  Skipping copy.


In [ ]:
evaluate_model(model, "Fine-Tuned Performance: Sentence Similarity", query, docs)


============================== Fine-Tuned Performance: Sentence Similarity ==============================
Original Sentence:  ሰማይ ምን አይነት ቀለም ነው?
Sentence | Similarity Score
ሰማይ ሰማያዊ ቀለም አለው። |  0.859 
አየር በምድር ዙሪያ ያለ ነው። |  0.650 
ባሕር ምን አይነት ቀለም ነው? |  0.622 
እኔ ምሳ እንጀራ በላሁ። |  0.167 


In [ ]:
model.push_to_hub("xlm-r-retrieval-am-v2")

ValueError: Invalid metadata in README.md.
- "license" must be one of [apache-2.0, mit, openrail, bigscience-openrail-m, creativeml-openrail-m, bigscience-bloom-rail-1.0, bigcode-openrail-m, afl-3.0, artistic-2.0, bsl-1.0, bsd, bsd-2-clause, bsd-3-clause, bsd-3-clause-clear, c-uda, cc, cc0-1.0, cc-by-2.0, cc-by-2.5, cc-by-3.0, cc-by-4.0, cc-by-sa-3.0, cc-by-sa-4.0, cc-by-nc-2.0, cc-by-nc-3.0, cc-by-nc-4.0, cc-by-nd-4.0, cc-by-nc-nd-3.0, cc-by-nc-nd-4.0, cc-by-nc-sa-2.0, cc-by-nc-sa-3.0, cc-by-nc-sa-4.0, cdla-sharing-1.0, cdla-permissive-1.0, cdla-permissive-2.0, wtfpl, ecl-2.0, epl-1.0, epl-2.0, etalab-2.0, eupl-1.1, agpl-3.0, gfdl, gpl, gpl-2.0, gpl-3.0, lgpl, lgpl-2.1, lgpl-3.0, isc, lppl-1.3c, ms-pl, apple-ascl, mpl-2.0, odc-by, odbl, openrail++, osl-3.0, postgresql, ofl-1.1, ncsa, unlicense, zlib, pddl, lgpl-lr, deepfloyd-if-license, llama2, llama3, llama3.1, llama3.2, llama3.3, gemma, unknown, other, array]